In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Main libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#Processing libraries
from sklearn.preprocessing import StandardScaler,MinMaxScaler

# Model libraries
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB

# Testing libraries
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV, learning_curve
from sklearn.metrics import accuracy_score, log_loss, confusion_matrix, classification_report, roc_curve, auc

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')
sns.set(font_scale=1.8)

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/bda/dataset/finalePython.csv')
data=data.drop('countrydest',axis=1)
data=data.drop(['countryorig','airportdest'],axis=1)
data=data.drop(['airportorig','airline','statenamedest','regiondest','statenameorig','regionorig'],axis=1)
data['partofdaydep']=data['partofdaydep'].apply(lambda x: 1 if x=='PM' else 0)
data['partofdayarr']=data['partofdayarr'].apply(lambda x: 1 if x=='PM' else 0)


In [ ]:
ritardi=data[(data['carrierdelay']>0) | (data['weatherdelay']>0) | (data['nasdelay']>0) | (data['securitydelay']>0)]
ritardi['totaldelay']=np.nan
for index,row in ritardi.iterrows():
  ritardi.at[index,'totaldelay']=row['carrierdelay']+row['weatherdelay']+row['nasdelay']+row['securitydelay']+row['lateaircraftdelay']

ritardi.head()

,dayofweek,weekofmonth,day,month,partofdaydep,partofdayarr,deptime,airtime,crsarrtime,flightnum,...,taxiOut,carrierdelay,weatherdelay,nasdelay,securitydelay,lateaircraftdelay,statecodeorigid,statecodedestid,uniquecarrierid,totaldelay
0,4,1,3,1,1,1,1829,77,1925,3920,...,10,2,0,0,0,32,14,20,0,34.0
1,4,1,3,1,1,1,1937,230,1940,509,...,7,10,0,0,0,47,14,28,0,57.0
2,4,1,3,1,1,1,1644,107,1725,1333,...,8,8,0,0,0,72,14,9,0,80.0
3,4,1,3,1,1,1,1452,213,1625,675,...,8,3,0,0,0,12,14,2,0,15.0
5,4,1,3,1,1,1,1416,49,1435,54,...,5,12,0,0,0,25,32,20,0,37.0


In [ ]:
ritardi_senza_zero=ritardi
print(len(ritardi_senza_zero))
zero = ritardi_senza_zero[ritardi_senza_zero['uniquecarrierid']==0]
one = ritardi_senza_zero[ritardi_senza_zero['uniquecarrierid']==1]
two = ritardi_senza_zero[ritardi_senza_zero['uniquecarrierid']==2]
three = ritardi_senza_zero[ritardi_senza_zero['uniquecarrierid']==3]
four = ritardi_senza_zero[ritardi_senza_zero['uniquecarrierid']==4]
five = ritardi_senza_zero[ritardi_senza_zero['uniquecarrierid']==5]
six = ritardi_senza_zero[ritardi_senza_zero['uniquecarrierid']==6]
seven = ritardi_senza_zero[ritardi_senza_zero['uniquecarrierid']==7]
eight = ritardi_senza_zero[ritardi_senza_zero['uniquecarrierid']==8]
nine = ritardi_senza_zero[ritardi_senza_zero['uniquecarrierid']==9]
balanced_data = pd.concat([nine, zero.sample(len(nine)), one.sample(len(nine)), two.sample(len(nine)), three.sample(len(nine)), four.sample(len(nine)), five.sample(len(nine)), six.sample(len(nine)), seven.sample(len(nine)), eight.sample(len(nine)), nine.sample(len(nine))])
print(one.shape,two.shape,three.shape,four.shape,five.shape,six.shape,seven.shape,eight.shape,nine.shape)
print(balanced_data.shape)

407170
(67028, 28) (51166, 28) (46768, 28) (34532, 28) (26631, 28) (26805, 28) (27421, 28) (13391, 28) (8001, 28)
(88011, 28)


In [ ]:
rs = 42
# zeros = data[data['Diabetes_012']==0]
# ones = data[data['Diabetes_012']==1]
# twos = data[data['Diabetes_012']==2]

# balanced_data = pd.concat([ones, zeros.sample(len(ones)), twos.sample(len(ones))]) Multiclass

zeros = ritardi_senza_zero[ritardi_senza_zero['uniquecarrierid']==0]
ones = ritardi_senza_zero[ritardi_senza_zero['uniquecarrierid']==1]
two = ritardi_senza_zero[ritardi_senza_zero['uniquecarrierid']==2]
three = ritardi_senza_zero[ritardi_senza_zero['uniquecarrierid']==3]
four = ritardi_senza_zero[ritardi_senza_zero['uniquecarrierid']==4]
five = ritardi_senza_zero[ritardi_senza_zero['uniquecarrierid']==5]
six = ritardi_senza_zero[ritardi_senza_zero['uniquecarrierid']==6]
seven = ritardi_senza_zero[ritardi_senza_zero['uniquecarrierid']==7]
eight = ritardi_senza_zero[ritardi_senza_zero['uniquecarrierid']==8]
nine = ritardi_senza_zero[ritardi_senza_zero['uniquecarrierid']==9]

balanced_data = pd.concat([nine, zero.sample(len(nine),random_state=rs), one.sample(len(nine),random_state=rs), two.sample(len(nine),random_state=rs), three.sample(len(nine),random_state=rs), four.sample(len(nine),random_state=rs), five.sample(len(nine),random_state=rs), six.sample(len(nine),random_state=rs), seven.sample(len(nine),random_state=rs), eight.sample(len(nine),random_state=rs), nine.sample(len(nine),random_state=rs)])

# balanced_data = pd.concat([ones.sample(500, random_state=rs), zeros.sample(500, random_state=rs)])
d2 = pd.concat([ones.sample(500, random_state=rs-1), zeros.sample(500, random_state=rs-1)])
label = balanced_data['uniquecarrierid']

# Data Normalization
scaler = MinMaxScaler()
scaled_d = scaler.fit_transform(balanced_data[:])
balanced_data[:] = scaled_d

scaled_d2 = scaler.fit_transform(d2[:])
d2[:] = scaled_d2

balanced_data.shape

(88011, 28)

In [ ]:
d2.head()

,dayofweek,weekofmonth,day,month,partofdaydep,partofdayarr,deptime,airtime,crsarrtime,flightnum,...,taxiOut,carrierdelay,weatherdelay,nasdelay,securitydelay,lateaircraftdelay,statecodeorigid,statecodedestid,uniquecarrierid,totaldelay
321968,0.333333,0.25,0.266667,0.6,0.0,1.0,0.385132,0.348285,0.558624,0.264463,...,0.112195,0.018692,0.0,0.027778,0.0,0.000000,0.897959,0.959184,1.0,0.032946
318582,0.500000,0.25,0.300000,0.6,1.0,1.0,0.675346,0.313984,0.654206,0.080892,...,0.058537,0.442368,0.0,0.000000,0.0,0.000000,0.897959,0.918367,1.0,0.273256
170090,1.000000,0.75,0.766667,0.2,0.0,1.0,0.460731,0.554090,0.594732,0.619584,...,0.131707,0.049844,0.0,0.048611,0.0,0.000000,0.795918,0.428571,1.0,0.069767
73851,0.000000,0.75,0.666667,0.0,1.0,1.0,0.560269,0.401055,0.732370,0.306536,...,0.009756,0.000000,0.0,0.004630,0.0,0.108871,0.040816,0.265306,1.0,0.001938
165400,1.000000,0.50,0.533333,0.2,0.0,1.0,0.422932,0.802111,0.567120,0.386927,...,0.029268,0.018692,0.0,0.076389,0.0,0.000000,0.795918,0.897959,1.0,0.073643
